In [ ]:
import time, network, espnow, math
from machine import Pin, I2C

MAC_RECEPTOR = b'\xCC\xDB\xA7\x97\xA9\xAC' # MAC del receptor

# I2C setup
i2c_left = I2C(1, scl=Pin(18), sda=Pin(19), freq=400000)
i2c_right = I2C(0, scl=Pin(22), sda=Pin(21), freq=400000)

MPU = 0x68
try:
    i2c_left.writeto_mem(MPU, 0x6B, b'\x00')
    i2c_right.writeto_mem(MPU, 0x6B, b'\x00')
except:
    pass

def to_int(h, l):
    v = (h << 8) | l
    return v - 65536 if v & 0x8000 else v

def read_accel(i2c_dev):
    try:
        d = i2c_dev.readfrom_mem(MPU, 0x3B, 6)
        ax = to_int(d[0], d[1])
        ay = to_int(d[2], d[3])
        return ax, ay
    except:
        return None, None

BTN_ON = Pin(32, Pin.IN, Pin.PULL_UP)    # Encender
BTN_OFF = Pin(26, Pin.IN, Pin.PULL_UP)   # Apagar
LED = Pin(2, Pin.OUT) # Led De comunicasion

# Estados de botones (IGUAL)
btn_on_state = 0
btn_off_state = 0
last_btn_time = 0

# Puntos calibrados
CAL = {
    "neutral":   (1601, -2935),
    "adelante":  (-14059, -1943),
    "atras":     (17318,  -984),
    "izquierda": (-5221, -15465),
    "derecha":   (-4115, 14447)
}

# SISTEMA DIFUSO
class SistemaDifusoSimple:
    def __init__(self):
        self.cal_points = CAL

        # UMBRALES OPTIMIZADOS (IGUAL QUE TU CÓDIGO)
        self.Y_DERECHA_MIN = 8000
        self.Y_DERECHA_FUERTE = 10000
        self.X_DERECHA_MAX = -2000
        self.ZONA_NEUTRAL = 5000

        # Factores para derecha (IGUAL)
        self.PESO_Y_DERECHA = 1.8
        self.PESO_X_DERECHA = 0.5

        # Historial (SEPARADO POR SENSOR)
        self.historial = []
        self.ultimo_movimiento = "neutral"
        self.derecha_detectada = False

    def verificar_derecha(self, ax, ay):
        """Verificación ESPECÍFICA para derecha (IGUAL QUE TU CÓDIGO)"""
        if ay < self.Y_DERECHA_MIN:
            return False, f"Y bajo: {ay}"

        if ax > self.X_DERECHA_MAX:
            return False, f"X no negativo: {ax}"

        target_x, target_y = self.cal_points["derecha"]
        dx = (ax - target_x) * self.PESO_X_DERECHA
        dy = (ay - target_y) * self.PESO_Y_DERECHA
        distancia = math.sqrt(dx*dx + dy*dy)

        if distancia < 7000:
            return True, f"Derecha: X={ax}, Y={ay}, Dist={distancia:.0f}"

        return False, f"No derecha: Dist={distancia:.0f}"

    def clasificar_movimiento_simple(self, ax, ay):
        """Clasificación simple pero efectiva (IGUAL QUE TU CÓDIGO)"""
        if ax is None or ay is None:
            return "neutral"

        # 1. PRIMERO: Verificar si es DERECHA
        es_derecha, mensaje = self.verificar_derecha(ax, ay)

        if es_derecha:
            if not self.derecha_detectada:
                self.derecha_detectada = True
            return "derecha"
        else:
            self.derecha_detectada = False

        # 2. Verificar si está en zona NEUTRAL
        neutral_x, neutral_y = self.cal_points["neutral"]
        dist_neutral = abs(ax - neutral_x) + abs(ay - neutral_y)

        if dist_neutral < self.ZONA_NEUTRAL:
            return "neutral"

        # 3. Calcular distancias a todos los puntos
        mejor_movimiento = "neutral"
        menor_distancia = 1000000

        for nombre, (cx, cy) in self.cal_points.items():
            if nombre == "neutral":
                continue

            distancia = abs(ax - cx) + abs(ay - cy)

            if distancia < menor_distancia:
                menor_distancia = distancia
                mejor_movimiento = nombre

        # 4. Si la distancia es muy grande, mantener neutral
        if menor_distancia > 20000:
            return "neutral"

        # 5. Actualizar historial para estabilidad
        self.historial.append(mejor_movimiento)
        if len(self.historial) > 3:
            self.historial.pop(0)

        # 6. Si hay historial, verificar tendencia
        if len(self.historial) >= 2:
            conteo = {}
            for mov in self.historial:
                conteo[mov] = conteo.get(mov, 0) + 1

            mas_comun = max(conteo, key=conteo.get)

            if conteo[mas_comun] >= 2:
                self.ultimo_movimiento = mas_comun
                return mas_comun

        self.ultimo_movimiento = mejor_movimiento
        return mejor_movimiento

# Crear DOS sistemas difusos independientes
sistema_left = SistemaDifusoSimple()
sistema_right = SistemaDifusoSimple()

# ESP-NOW (IGUAL QUE TU CÓDIGO)
w = network.WLAN(network.STA_IF)
w.active(True)
e = espnow.ESPNow()
e.active(True)
try:
    e.add_peer(MAC_RECEPTOR)
except:
    pass

# Estado (IGUAL)
SISTEMA_ENCENDIDO = False
velocidad = 60
ultimo_comando = ""
movimiento_actual_left = ""
movimiento_actual_right = ""
last_sync = time.ticks_ms()

# Debounce (IGUAL)
DEBOUNCE_TIME = 400

def send_cmd(cmd_bytes):
    try:
        e.send(MAC_RECEPTOR, cmd_bytes)
        LED.on()
        time.sleep(0.02)
        LED.off()
        return True
    except:
        LED.off()
        return False

print("\n" + "="*60)
print("EMISOR - 2 Sensores (Uno por motor)")
print("="*60)
print("Estado: APAGADO")
print("Presiona ON (GPIO32) para iniciar")

while True:
    tiempo_actual = time.ticks_ms()
    tiempo_dif = time.ticks_diff(tiempo_actual, last_btn_time)

    # Botón ON (IGUAL)
    if not BTN_ON.value():
        if btn_on_state == 0 and tiempo_dif > DEBOUNCE_TIME:
            btn_on_state = 1
            last_btn_time = tiempo_actual

            if not SISTEMA_ENCENDIDO:
                SISTEMA_ENCENDIDO = True
                print("\n>>> SISTEMA ENCENDIDO <<<")
                print(f"Velocidad fija: {velocidad}%")
                send_cmd(b"confirm")
                ultimo_comando = ""
                movimiento_actual_left = ""
                movimiento_actual_right = ""
                sistema_left = SistemaDifusoSimple()
                sistema_right = SistemaDifusoSimple()
                time.sleep(0.5)
    else:
        btn_on_state = 0

    # Botón OFF (IGUAL)
    if not BTN_OFF.value():
        if btn_off_state == 0 and tiempo_dif > DEBOUNCE_TIME:
            btn_off_state = 1
            last_btn_time = tiempo_actual

            if SISTEMA_ENCENDIDO:
                SISTEMA_ENCENDIDO = False
                print("\n>>> SISTEMA APAGADO <<<")
                send_cmd(b"OFF")
                ultimo_comando = ""
                movimiento_actual_left = ""
                movimiento_actual_right = ""
                time.sleep(0.5)
    else:
        btn_off_state = 0

    # Sistema APAGADO (IGUAL)
    if not SISTEMA_ENCENDIDO:
        if (tiempo_actual // 1000) % 2 == 0:
            LED.on()
        else:
            LED.off()
        time.sleep(0.1)
        continue

    # SYNC (IGUAL)
    if time.ticks_diff(tiempo_actual, last_sync) > 500:
        try:
            e.send(MAC_RECEPTOR, b"SYNC")
        except:
            pass
        last_sync = tiempo_actual

    # Leer AMBOS IMUs (CAMBIO PRINCIPAL: DOS LECTURAS)
    ax_left, ay_left = read_accel(i2c_left)
    ax_right, ay_right = read_accel(i2c_right)

    if ax_left is None or ax_right is None:
        time.sleep(0.05)
        continue

    # Clasificar CADA SENSOR INDEPENDIENTEMENTE
    movimiento_left = sistema_left.clasificar_movimiento_simple(ax_left, ay_left)
    movimiento_right = sistema_right.clasificar_movimiento_simple(ax_right, ay_right)

    # Crear comando COMBINADO (CAMBIO PRINCIPAL)
    comando_combinado = f"LEFT:{movimiento_left}:{velocidad}|RIGHT:{movimiento_right}:{velocidad}"

    # Enviar comando si cambió (CAMBIO: comparar ambos)
    if movimiento_left != movimiento_actual_left or movimiento_right != movimiento_actual_right:
        movimiento_actual_left = movimiento_left
        movimiento_actual_right = movimiento_right

        if movimiento_left != "neutral" or movimiento_right != "neutral":
            print(f"→ LEFT:{movimiento_left.upper()} RIGHT:{movimiento_right.upper()}")
            send_cmd(comando_combinado.encode())
            ultimo_comando = comando_combinado
        elif ultimo_comando and "neutral" not in ultimo_comando:
            print("→ NEUTRAL (ambos)")
            send_cmd(b"NEUTRAL")
            ultimo_comando = ""

    time.sleep(0.08)